<a href="https://colab.research.google.com/github/poliakovfm/itmo_transcriptomics_homeworks/blob/main/HW0_Trans_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

In [13]:
import pandas as pd
from Bio.Seq import Seq
import warnings
warnings.simplefilter('ignore')

In [4]:
# function that finds all ORFs in a given sequence
# problem inspired by https://rosalind.info/problems/orf/

def find_all_orfs():

  # принимаем строку на вход
  input_sequence = input()

  # создаем дф со строками в разных направлениях
  sequences = pd.DataFrame({
      'forward': [input_sequence],
      'reverse': [input_sequence[::-1]],
      'complement': [str(Seq(input_sequence).complement())],
      'reverse complement': [str(Seq(input_sequence).reverse_complement())]
  })

  # создаем дф, куда будем записывать результаты
  orf_df = pd.DataFrame(columns=['orf_nt_sequence', 'orf_aa_sequence', 'start_position', 'end_position', 'nt_length', 'aa_length', 'direction'])

  # задаем старт и стоп кодоны
  start_codon = 'ATG'
  stop_codons = ['TAA', 'TAG', 'TGA']


  #обходим все направления строки
  for i, sequence in enumerate(list(sequences.iloc[0])):

    # создаем два пустых списка, куда мы будем записывать координаты начала соответствущих кодонов
    start_codons_positions = []
    stop_codons_positions = []

    # заполняем эти списки
    start_codons_positions = [i for i in range(len(sequence)) if sequence.startswith(start_codon, i)]

    for stop_codon in stop_codons:
      single_codon_positions = [i for i in range(len(sequence)) if sequence.startswith(stop_codon, i)]
      stop_codons_positions += single_codon_positions


    for start_codon_position in start_codons_positions:
      for stop_codon_position in stop_codons_positions:

        # 1 условие: координата старт кодона лежит раньше координаты стоп кодона
        if (start_codon_position < stop_codon_position

            # 2 условие: данный стоп кодон — наименьший из множества всех стоп кодонов, которые больше старта
            and stop_codon_position == min([k for k in stop_codons_positions if k > 32])

            # 3 условие: старт и стоп лежат на одной рамке считывания (3 добавляем, если старт лежит на 0 координате)
            and (start_codon_position +3) % 3 == start_codon_position % 3):

          # делаем срез строки по соответствующим координатам
          orf_sequence =  sequence[start_codon_position:stop_codon_position]

          # заполняем наш дф соответствующими значениями
          orf_df = orf_df.append(pd.Series(), ignore_index=True)
          orf_df.iloc[len(orf_df.index)-1].loc['orf_nt_sequence'] = orf_sequence
          orf_df.iloc[len(orf_df.index)-1].loc['orf_aa_sequence'] = str(Seq(orf_sequence).translate())
          orf_df.iloc[len(orf_df.index)-1].loc['start_position'] = start_codon_position
          orf_df.iloc[len(orf_df.index)-1].loc['end_position'] = stop_codon_position
          orf_df.iloc[len(orf_df.index)-1].loc['nt_length'] = len(orf_sequence)
          orf_df.iloc[len(orf_df.index)-1].loc['aa_length'] = len(str(Seq(orf_sequence).translate()))
          orf_df.iloc[len(orf_df.index)-1].loc['direction'] = sequences.columns[i]




  return orf_df


In [14]:
# running the function and saving its output to new_df
# test input string: "AGCCTTGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

new_df = find_all_orfs()

AGCCTTGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG


In [15]:
# checking its contents

new_df

,orf_nt_sequence,orf_aa_sequence,start_position,end_position,nt_length,aa_length,direction
0,ATGGGGATGACCCCGCGACTTGGAT,MGMTPRLG,24,49,25,8,forward
1,ATGACCCCGCGACTTGGAT,MTPRLG,30,49,19,6,forward
2,ATGAGCCTAGTAAGTCCGAATAAGGTTTTCTC,MSLVSPNKVF,9,41,32,10,reverse
3,ATGTACCCCTACTGGGGCGC,MYPYWG,21,41,20,6,complement
4,ATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTC,MLLGSFRLIPKET,5,45,40,13,reverse complement
